# MeerKAT conversion guide

In [1]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")


# These are the three API functions that are currently supported
from xradio.image import load_image, read_image, write_image

XRADIO version 0.0.31 already installed.


## Download dataset

In [2]:
import graphviper

graphviper.utils.data.download(file="small_meerkat.ms")

[2024-07-18 14:49:22,300]  WARNING  graphviper:  File exists: /Users/jsteeb/Dropbox/graphviper/src/graphviper/utils/data/.dropbox 
[2024-07-18 14:49:22,301]     INFO  graphviper:  Updating file metadata information ...  
 

 Download List     
 ────────────────── 
  small_meerkat.ms

[2024-07-18 14:49:23,051]     INFO  graphviper:  File exists: small_meerkat.ms 


## Convert to Processing Set

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

ms_file = "small_meerkat.ms"
main_chunksize = {"frequency": 1, "time": 20}  # baseline, polarization
outfile = "small_meerkat.vis.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    parallel=False,
    overwrite=True,
    main_chunksize=main_chunksize,
)

[2024-07-18 14:49:23,252]     INFO  graphviper:  Partition scheme that will be used: ['DATA_DESC_ID', 'OBS_MODE', 'FIELD_ID'] 
[2024-07-18 14:49:23,263]     INFO  graphviper:  Number of partitions: 3 
[2024-07-18 14:49:23,263]     INFO  graphviper:  DDI [0], STATE [1], FIELD [0], SCAN [1] 
[2024-07-18 14:49:23,682]     INFO  graphviper:  DDI [0], STATE [2], FIELD [1], SCAN [2 4 6] 
[2024-07-18 14:49:24,013]     INFO  graphviper:  DDI [0], STATE [3], FIELD [2], SCAN [3 5] 


## Processing Set

In [4]:
from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set(ps_store=outfile)
ps.summary()

,name,obs_mode,shape,polarization,spw_id,field_id,field_name,source_id,source_name,field_coords,start_frequency,end_frequency
0,small_meerkat_1,"CALIBRATE_PHASE,CALIBRATE_AMPLI","(42, 6, 50, 4)","[XX, XY, YX, YY]",0,[1],[J1619-8418],[1],[J1619-8418],"[fk5, 16h19m33.97s, -84d18m19.10s]",3.265869e+09,3.276337e+09
1,small_meerkat_0,"CALIBRATE_BANDPASS,CALIBRATE_FLUX","(74, 6, 50, 4)","[XX, XY, YX, YY]",0,[0],[J1939-6342],[0],[J1939-6342],"[fk5, 19h39m25.03s, -63d42m45.60s]",3.265869e+09,3.276337e+09
2,small_meerkat_2,TARGET,"(223, 6, 50, 4)","[XX, XY, YX, YY]",0,[2],[J0358-8103],[2],[J0358-8103],"[fk5, 3h58m31.50s, -81d03m45.70s]",3.265869e+09,3.276337e+09


In [5]:
ps.get(0).attrs["antenna_xds"]

<xarray.Dataset> Size: 516B
Dimensions:        (antenna_id: 3, xyz_label: 3)
Coordinates:
  * antenna_id     (antenna_id) int64 24B 0 1 2
    mount          (antenna_id) <U6 72B dask.array<chunksize=(3,), meta=np.ndarray>
    name           (antenna_id) <U4 48B dask.array<chunksize=(3,), meta=np.ndarray>
    station        (antenna_id) <U4 48B dask.array<chunksize=(3,), meta=np.ndarray>
    type           (antenna_id) <U12 144B dask.array<chunksize=(3,), meta=np.ndarray>
  * xyz_label      (xyz_label) <U1 12B 'x' 'y' 'z'
Data variables:
    DISH_DIAMETER  (antenna_id) float64 24B dask.array<chunksize=(3,), meta=np.ndarray>
    FEED_OFFSET    (antenna_id, xyz_label) float64 72B dask.array<chunksize=(3, 3), meta=np.ndarray>
    POSITION       (antenna_id, xyz_label) float64 72B dask.array<chunksize=(3, 3), meta=np.ndarray>